In [181]:
import biblio_modules

import time
from datetime import *
import calendar


import pyarrow as py
import pandas as pd
import numpy as np
from pytz import timezone

In [182]:
transactions_processing = pd.read_csv("../Source/Transactions.csv", sep=";")
transactions_processing

,id_prod,date,session_id,client_id
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033
...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848


In [183]:
transactions_processing[['Date','Time']] = transactions_processing['date'].str.split(expand=True)
transactions_processing['Date'] = pd.to_datetime(transactions_processing['Date'])

In [184]:
transactions_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     687534 non-null  object        
 1   date        687534 non-null  object        
 2   session_id  687534 non-null  object        
 3   client_id   687534 non-null  object        
 4   Date        687534 non-null  datetime64[ns]
 5   Time        687534 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 31.5+ MB


In [185]:
transactions_processing.loc[transactions_processing['Time'].str.startswith('24')]

,id_prod,date,session_id,client_id,Date,Time
198325,1_400,2021-10-02 24:04:58.422464,s_107311,c_2513,2021-10-02,24:04:58.422464
198326,1_601,2021-10-02 24:04:59.254020,s_107311,c_2513,2021-10-02,24:04:59.254020
198327,1_735,2021-10-02 24:04:59.422464,s_107311,c_2513,2021-10-02,24:04:59.422464
198328,1_351,2021-10-02 24:04:59.913108,s_107311,c_2513,2021-10-02,24:04:59.913108
198329,1_400,2021-10-02 24:06:02.588166,s_107205,c_5036,2021-10-02,24:06:02.588166
...,...,...,...,...,...,...
221108,1_445,2021-10-26 24:36:01.240212,s_105041,c_3949,2021-10-26,24:36:01.240212
221109,1_296,2021-10-26 24:36:21.240212,s_105041,c_3949,2021-10-26,24:36:21.240212
221820,1_45,2021-10-27 24:18:21.505552,s_98882,c_7656,2021-10-27,24:18:21.505552
221821,1_431,2021-10-27 24:20:46.744883,s_99594,c_8073,2021-10-27,24:20:46.744883


In [186]:
transactions_processing.loc[transactions_processing['Time'].str.startswith('24'), 'Date'] = transactions_processing.loc[transactions_processing['Time'].str.startswith('24'), 'Date'] + pd.DateOffset(days = 1)
transactions_processing

,id_prod,date,session_id,client_id,Date,Time
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,2021-03-01,00:01:07.843138
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,2021-03-01,00:02:26.047414
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,2021-03-01,00:02:38.311413
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,2021-03-01,00:04:54.559692
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,2021-03-01,00:05:18.801198
...,...,...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573,2023-02-28,23:49:03.148402
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50,2023-02-28,23:51:29.318531
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488,2023-02-28,23:53:18.929676
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848,2023-02-28,23:58:00.107815


In [187]:
condition = transactions_processing['Time'].str.startswith('24')
transactions_processing.loc[condition, 'Time'] = '00' + transactions_processing.loc[condition, 'Time'].str[2:]
transactions_processing[transactions_processing['date'].str.contains(' 24:')]


,id_prod,date,session_id,client_id,Date,Time
198325,1_400,2021-10-02 24:04:58.422464,s_107311,c_2513,2021-10-03,00:04:58.422464
198326,1_601,2021-10-02 24:04:59.254020,s_107311,c_2513,2021-10-03,00:04:59.254020
198327,1_735,2021-10-02 24:04:59.422464,s_107311,c_2513,2021-10-03,00:04:59.422464
198328,1_351,2021-10-02 24:04:59.913108,s_107311,c_2513,2021-10-03,00:04:59.913108
198329,1_400,2021-10-02 24:06:02.588166,s_107205,c_5036,2021-10-03,00:06:02.588166
...,...,...,...,...,...,...
221108,1_445,2021-10-26 24:36:01.240212,s_105041,c_3949,2021-10-27,00:36:01.240212
221109,1_296,2021-10-26 24:36:21.240212,s_105041,c_3949,2021-10-27,00:36:21.240212
221820,1_45,2021-10-27 24:18:21.505552,s_98882,c_7656,2021-10-28,00:18:21.505552
221821,1_431,2021-10-27 24:20:46.744883,s_99594,c_8073,2021-10-28,00:20:46.744883


In [188]:
transactions_processing['Date'] = transactions_processing['Date'].astype(str)
transactions_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id_prod     687534 non-null  object
 1   date        687534 non-null  object
 2   session_id  687534 non-null  object
 3   client_id   687534 non-null  object
 4   Date        687534 non-null  object
 5   Time        687534 non-null  object
dtypes: object(6)
memory usage: 31.5+ MB


In [198]:
transactions_processing['Timestamp'] = transactions_processing['Date'] + ' ' + transactions_processing['Time']
transactions_processing

,id_prod,date,session_id,client_id,Date,Time,Timestamp
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,2021-03-01,00:01:07.843138,2021-03-01 00:01:07.843138
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,2021-03-01,00:02:26.047414,2021-03-01 00:02:26.047414
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,2021-03-01,00:02:38.311413,2021-03-01 00:02:38.311413
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,2021-03-01,00:04:54.559692,2021-03-01 00:04:54.559692
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,2021-03-01,00:05:18.801198,2021-03-01 00:05:18.801198
...,...,...,...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573,2023-02-28,23:49:03.148402,2023-02-28 23:49:03.148402
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50,2023-02-28,23:51:29.318531,2023-02-28 23:51:29.318531
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488,2023-02-28,23:53:18.929676,2023-02-28 23:53:18.929676
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848,2023-02-28,23:58:00.107815,2023-02-28 23:58:00.107815


In [199]:
transactions_processing['Timestamp'] = pd.to_datetime(transactions_processing['Timestamp'], errors='raise')
transactions_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687534 entries, 0 to 687533
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   id_prod     687534 non-null  object        
 1   date        687534 non-null  object        
 2   session_id  687534 non-null  object        
 3   client_id   687534 non-null  object        
 4   Date        687534 non-null  object        
 5   Time        687534 non-null  object        
 6   Timestamp   687534 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 36.7+ MB


In [200]:
transactions_processing.drop(columns=['date', 'Date', 'Time'])

,id_prod,session_id,client_id,Timestamp
0,0_1259,s_1,c_329,2021-03-01 00:01:07.843138
1,0_1390,s_2,c_664,2021-03-01 00:02:26.047414
2,0_1352,s_3,c_580,2021-03-01 00:02:38.311413
3,0_1458,s_4,c_7912,2021-03-01 00:04:54.559692
4,0_1358,s_5,c_2033,2021-03-01 00:05:18.801198
...,...,...,...,...
687529,1_508,s_348444,c_3573,2023-02-28 23:49:03.148402
687530,2_37,s_348445,c_50,2023-02-28 23:51:29.318531
687531,1_695,s_348446,c_488,2023-02-28 23:53:18.929676
687532,0_1547,s_348447,c_4848,2023-02-28 23:58:00.107815


In [88]:
#france_tz = timezone('Europe/Paris')
#timestamp = pd.Timestamp('2023-09-28 24:21:23.82736', year=2023, month=9, day=28, hour=24, minute=21, second=23, microsecond=82736, fold=0, )


In [89]:
#timestamp = pd.Timestamp('2023-09-28 24:21:23.82736')
#shifted_timestamp = timestamp - pd.Timedelta(hours=1)
#print(shifted_timestamp) 

In [113]:
transactions_processing

,id_prod,date,session_id,client_id,Date,Time
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,2021-03-01,00:01:07.843138
1,0_1390,2021-03-01 00:02:26.047414,s_2,c_664,2021-03-01,00:02:26.047414
2,0_1352,2021-03-01 00:02:38.311413,s_3,c_580,2021-03-01,00:02:38.311413
3,0_1458,2021-03-01 00:04:54.559692,s_4,c_7912,2021-03-01,00:04:54.559692
4,0_1358,2021-03-01 00:05:18.801198,s_5,c_2033,2021-03-01,00:05:18.801198
...,...,...,...,...,...,...
687529,1_508,2023-02-28 23:49:03.148402,s_348444,c_3573,2023-02-28,23:49:03.148402
687530,2_37,2023-02-28 23:51:29.318531,s_348445,c_50,2023-02-28,23:51:29.318531
687531,1_695,2023-02-28 23:53:18.929676,s_348446,c_488,2023-02-28,23:53:18.929676
687532,0_1547,2023-02-28 23:58:00.107815,s_348447,c_4848,2023-02-28,23:58:00.107815


In [91]:
# première timestamp enregistrer le 1er mars 2021
# Je dois vérifier quelle heure est le temps standard en france
# D'après mes recherche, c'est l'heure d'hiver qui est la bonne (Dernier dimanche d'octobre -> dernier dimanche de mars)
# Comme le premier enregistrement du site à lieu le 1er mars 2021, c'est bien un horaire standard

In [114]:
transactions_processing.head(1)

,id_prod,date,session_id,client_id,Date,Time
0,0_1259,2021-03-01 00:01:07.843138,s_1,c_329,2021-03-01,00:01:07.843138


In [115]:
#Je cherche l'index qui à été enregistrer en dernier le 28 mars 2021 à 01:59:59 max
transactions_processing[transactions_processing['date'].str.contains('2021-03-28 01')].tail(1)# et je regarde l'index du dernier enregistrement

,id_prod,date,session_id,client_id,Date,Time
25063,0_1410,2021-03-28 01:58:39.208172,s_12429,c_7433,2021-03-28,01:58:39.208172


In [116]:
#J'affiche également l'index qui suit pour vérifier qu'il y a bien eu changement d'heure
transactions_processing.iloc[[25063, 25065]]

,id_prod,date,session_id,client_id,Date,Time
25063,0_1410,2021-03-28 01:58:39.208172,s_12429,c_7433,2021-03-28,01:58:39.208172
25065,0_1163,2021-03-28 03:01:18.162842,s_12431,c_3449,2021-03-28,03:01:18.162842


In [95]:
#Il y a bien eu changement d'heure
#je dois donc réculer d'1h tout les enregistrement entre le 28 mars 2021 à 2h du matin et le 31 octobre 2021 à 3h du matin

In [117]:
#On voit qu'il y a deux fois plus d'enregistrement que les autres heures 
transactions_processing[transactions_processing['date'].str.contains('2021-10-31 01')].tail(1)# et je regarde l'index du dernier enregistrement

,id_prod,date,session_id,client_id,Date,Time
224941,0_1397,2021-10-31 01:59:13.566641,s_112514,c_2920,2021-10-31,01:59:13.566641


In [127]:
h_été = transactions_processing.iloc[25065:224942]
h_été.loc[h_été['Time'].str.startswith('24')]

/var/folders/rg/h0w5cjr90257kx52nm8zrv280000gn/T/ipykernel_4556/3274815966.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_été['Date'] = pd.to_datetime(h_été['Date'])


,id_prod,date,session_id,client_id,Date,Time
25065,0_1163,2021-03-28 03:01:18.162842,s_12431,c_3449,2021-03-28,03:01:18.162842
25066,1_480,2021-03-28 03:03:01.094750,s_12432,c_5546,2021-03-28,03:03:01.094750
25067,0_1583,2021-03-28 03:03:12.162238,s_12430,c_7350,2021-03-28,03:03:12.162238
25068,0_1982,2021-03-28 03:03:55.678602,s_12433,c_4367,2021-03-28,03:03:55.678602
25069,0_1882,2021-03-28 03:06:16.034177,s_12434,c_8036,2021-03-28,03:06:16.034177
...,...,...,...,...,...,...
224937,0_1156,2021-10-31 01:56:31.522509,s_112512,c_8128,2021-10-31,01:56:31.522509
224938,0_1132,2021-10-31 01:57:01.820377,s_112513,c_3381,2021-10-31,01:57:01.820377
224939,0_1142,2021-10-31 01:57:40.196831,s_112507,c_4857,2021-10-31,01:57:40.196831
224940,0_1078,2021-10-31 01:57:40.950446,s_112508,c_6390,2021-10-31,01:57:40.950446


In [120]:
#je localise les lignes d'heures d'été 

#transactions_processing[transactions_processing.iloc[25065:224942, 1]]
#timestamp = pd.Timestamp(heure_été['date'], )
#shifted_timestamp = timestamp - pd.Timedelta(hours=1)

,id_prod,date,session_id,client_id,Date,Time
198325,1_400,2021-10-02 24:04:58.422464,s_107311,c_2513,2021-10-02,24:04:58.422464
198326,1_601,2021-10-02 24:04:59.254020,s_107311,c_2513,2021-10-02,24:04:59.254020
198327,1_735,2021-10-02 24:04:59.422464,s_107311,c_2513,2021-10-02,24:04:59.422464
198328,1_351,2021-10-02 24:04:59.913108,s_107311,c_2513,2021-10-02,24:04:59.913108
198329,1_400,2021-10-02 24:06:02.588166,s_107205,c_5036,2021-10-02,24:06:02.588166
...,...,...,...,...,...,...
221108,1_445,2021-10-26 24:36:01.240212,s_105041,c_3949,2021-10-26,24:36:01.240212
221109,1_296,2021-10-26 24:36:21.240212,s_105041,c_3949,2021-10-26,24:36:21.240212
221820,1_45,2021-10-27 24:18:21.505552,s_98882,c_7656,2021-10-27,24:18:21.505552
221821,1_431,2021-10-27 24:20:46.744883,s_99594,c_8073,2021-10-27,24:20:46.744883


In [98]:
#Les ligne qui dépassent l'heure réglementaire de 23:59:59
#transactions_processing[transactions_processing['date'].str.contains(' 24:')]
#Ca veut dire que ces heures devrait être reculer d'1heure

In [99]:
#Je dois trouver la date et l'heure à laquelle il y a eu un saut d'1 heure (entre 2 et 3h du matin)
